<a href="https://colab.research.google.com/github/Cehiim/TeoriaDosGrafos/blob/main/ProjetoFinal/ProjetoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informações gerais

## Tema
* Aplicação com busca semântica para representação de grafos.

## Integrantes
* Cesar Hideki Imai - 10402758.
* João Victor Dallapé Madeira - 10400725.
* David Varão Lima Bentes Pessoa - 10402647.
* André Franco Ranieri - 10390470.

# Setup

## Integração dos pacotes

O pacote `vectordb2` é usado para armazenar e recuperar textos usando técnicas de *chunking* (segmentação de texto), *embedding* (conversão de texto para vetores numéricos) e busca vetorial.

[Referência](https://vectordb.com/)

In [68]:
%pip install vectordb2

O pacote `requests` é ser usado para recuperar o arquivo por meio de requisição em HTTP.

In [69]:
%pip install requests

O pacote `network pyvis` é usado para criar uma visualização interativa de grafos.

[Referência](https://pyvis.readthedocs.io/en/latest/documentation.html)

In [70]:
%pip install networkx pyvis

## Importação das bibliotecas

In [71]:
from vectordb import Memory
import requests
from pyvis.network import Network
import os # Será usado métodos para limpar o terminal para atualizar a interface em cada iteração do sistema
import time # Será usado método de espera para atualizar a interface gradualmente
from ipywidgets import widgets # Será usado para criar interfaces de usuário interativas com widgets
from IPython.display import display, clear_output #Será usado para mostrar interações visuais de maneira mais sofisticada

## Classe GrafoNDR

In [72]:
# Grafo como uma matriz de adjacência não-direcionado rotulado
class GrafoNDR(): # Ex 8
    TAM_MAX_DEFAULT = 100 # qtde de vértices máxima default
    # construtor da classe grafo
    def __init__(self, n=TAM_MAX_DEFAULT):
        self.n = n # número de vértices
        self.m = 0 # número de arestas
        # matriz de adjacência
        self.adj = [[0 for i in range(n)] for j in range(n)]

    def insereA(self, v, w, p):
        if(v == w):
            return
        if self.adj[v][w] == 0:
            self.adj[v][w] = p
            self.adj[w][v] = p
            self.m += 1  # atualiza qtd arestas

# remove uma aresta v->w do Grafo
    def removeA(self, v, w):
        if(v == w):
            return
        # testa se temos a aresta
        if self.adj[v][w] != 0:
            self.adj[v][w] = 0
            self.adj[w][v] = 0
            self.m -= 1  # atualiza qtd arestas

    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f"Adj[{i:2d},{w:2d}] = {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f" {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

    def insereV(self):
        for i in range(self.n):
            self.adj[i].append(0)
        self.n += 1
        self.adj.append([0]*self.n)

    def removeV(self, vertice):
        if(vertice >= self.n or vertice < 0):
            return False
        for i in range(self.n):
            if(i == vertice):
                for j in range(self.n):
                    self.removeA(vertice, j) # Exclui arestas fantasmas
            self.removeA(i,vertice)
            if(i >= vertice and i != self.n-1): # Substitui as conexões do vértice a ser retirado e
                              # os vértices posteriores a ele com as conexões do próximo vértice
                self.adj[i] = self.adj[i+1]
            self.adj[i].pop(vertice) # Remove o vértice escolhido da linha da matriz
        self.adj.pop() # Remove a última linha da matriz
        self.n -= 1
        return True

    def dfs(self, visitados, vertice): # Depth First Search
        visitados[vertice] = True
        for i in range(self.n):
            if(self.adj[vertice][i] != 0 and visitados[i] == False): # Caso haja acesso para um próximo vértice que não foi visitado
                self.dfs(visitados, i)

    def conexidade(self):
        for i in range(self.n):
            visitados = [False] * self.n
            self.dfs(visitados, i)
            if(all(visitados)): # Caso todos tenham sido visitados
                return True
        return False

## Funções auxiliares

### Cria vértice

In [73]:
def criaVertice(palavra, indice):
  vertice = {
      "palavra": palavra, # Recupera cada palavra e tira o "\n"
      "indice": indice,
      "n_vizinhos": 0,
      "proximos": []
  }
  return vertice

### Lê arquivo

Os dados do documento são importados e guardados na variável `dados`.

In [74]:
def leArquivoHTTP(url):
  arquivo = requests.get(url).text

  lista = arquivo.split() # Distribui cada elemento do arquivo numa lista
  n_palavras = int(lista.pop(0)) # Separa o número de palavras (primeira linha do arquivo)
  vertices = []
  for i in range(n_palavras):
    vertice = criaVertice(lista[i], i)
    vertices.append(vertice)

  dados = [n_palavras]
  dados.append(vertices)

  return dados

In [75]:
def leArquivo(origem):
  try:
    with open(origem, 'r', encoding='utf-8') as arquivo:
      n_palavras = int(arquivo.readline()) # Recupera o número de palavras (primeira linha do arquivo)

      vertices = []
      for i in range(n_palavras):
        vertice = criaVertice(arquivo.readline().strip(), i)
        vertices.append(vertice)

    dados = [n_palavras]
    dados.append(vertices)

    return dados

  except FileNotFoundError:
      print("[Erro: Arquivo não encontrado]")

### Embedding

Cada palavra é convertida para um vetor numérico e guardada na memória.

In [76]:
def embedding(memoria, n_palavras, vertices): # Método para fazer o embedding e inserção na memória de todas as palavras
  for i in range(n_palavras):
    memoria.save(vertices[i]["palavra"])

### Busca vetorial

Quanto menor é a distância, maior é a proximidade semântica.

In [77]:
def buscaVetorial(memoria, palavra): # Método para retornar os quatro elementos com maior proximidade semântica de uma palavra
  busca = memoria.search(palavra, top_n=4)
  return busca

### Busca índice

Busca um índice através da palavra dentro da lista de vértices.

In [78]:
def buscaIndice(n_palavras, vertices, palavra):
  for i in range(n_palavras):
    if(vertices[i]["palavra"] == palavra):
      return vertices[i]["indice"]
  return -1

### Busca palavra

Busca uma palavra através do índice dentro da lista de vértices.

In [79]:
def buscaPalavra(n_palavras, vertices, indice):
  if(indice >= n_palavras or indice < 0):
    return "[Erro: índice inválido]"
  for i in range(n_palavras):
    if(vertices[i]["indice"] == indice):
      return vertices[i]["palavra"]
  return "[Erro: índice não encontrado]"

### Cria vizinho

In [80]:
def criaVizinho(palavra, indice, distancia):
  proximo = {
      "vizinho": palavra,
      "indice": indice,
      "distancia": distancia
  }
  return proximo

### Integra grafo

A palavra mais próxima armazenada na memória é ela mesma, portanto para encontrar as outras três palavras mais próximas foram recuperadas as palavras de índice 1 até 4.

In [81]:
def integraGrafo(memoria, n_palavras, vertices):
  grafo = GrafoNDR(n_palavras) # Cria um grafo rotulado
  for i in range(n_palavras):
    busca = buscaVetorial(memoria, vertices[i]["palavra"])
    proximos = []
    for j in range(1,4):
      palavra = busca[j]['chunk']
      distancia = busca[j]['distance']
      proximo = criaVizinho(palavra, buscaIndice(n_palavras, vertices, palavra), distancia)
      proximos.append(proximo)
      vertices[i]["n_vizinhos"] += 1
      grafo.insereA(vertices[i]["indice"], proximo["indice"], proximo["distancia"])
    vertices[i]["proximos"] = proximos
  return grafo

### Grava dados

In [82]:
def gravaDados(n_palavras, vertices):
  with open("grafo.txt", "w") as arquivo:
    for i in range(n_palavras):
      palavra = vertices[i]["palavra"]
      arquivo.write(palavra+"\n")

### Insere vértice

In [83]:
def insereVertice(grafo, n_palavras, vertices, palavra):
  if(buscaIndice(n_palavras, vertices, palavra) == -1):
    grafo.insereV()
    vertices.append(criaVertice(palavra, n_palavras))
  else:
    print("[Erro: Palavra já existe]")

### Cria aresta

In [84]:
def criaAresta(origem, destino):
  aresta = [0] * 2
  aresta[0] = origem
  aresta[1] = destino
  return aresta

### Busca vizinho

In [85]:
def buscaVizinho(n_vizinhos, vizinhos, indice):
  for i in range(n_vizinhos):
    if(vizinhos[i]["indice"] == indice):
      return i
  return -1

### Insere aresta

In [86]:
def insereAresta(grafo, n_palavras, vertices, aresta, peso):
  origem = aresta[0]
  destino = aresta[1]
  if(origem >= n_palavras or origem < 0):
    print("[Erro: origem não existe]")

  elif(destino >= n_palavras or destino < 0):
    print("[Erro: destino não existe]")

  else:
    n_vizinhos = vertices[origem]["n_vizinhos"]
    vizinhos = vertices[origem]["proximos"]

    if(buscaVizinho(n_vizinhos, vizinhos, destino) != -1):
      print("[Erro: aresta já existe]")

    else:
      grafo.insereA(origem, destino, peso)
      palavra_destino = buscaPalavra(n_palavras, vertices, destino)
      vertices[origem]["proximos"].append(criaVizinho(palavra_destino, destino, peso))
      vertices[origem]["n_vizinhos"] += 1

### Remove aresta

In [87]:
def removeAresta(grafo, n_palavras, vertices, aresta):
  origem = aresta[0]
  destino = aresta[1]
  if(origem >= n_palavras or origem < 0):
    print("[Erro: origem não existe]")

  elif(destino >= n_palavras or destino < 0):
    print("[Erro: destino não existe]")

  else:
    n_vizinhos = vertices[origem]["n_vizinhos"]
    vizinhos = vertices[origem]["proximos"]
    posicao = buscaVizinho(n_vizinhos, vizinhos, destino)
    if(posicao == -1):
      print("[Erro: aresta não existe]")

    else:
      grafo.removeA(origem, destino)
      vertices[origem]["proximos"].pop(posicao)
      vertices[origem]["n_vizinhos"] -= 1

### Remove vértice

In [88]:
def removeVertice(grafo, n_palavras, vertices, removido):
  if(removido >= n_palavras or removido < 0):
    print("[Erro: vértice não existe]")
  else:
    grafo.removeV(removido)
    for i in range(n_palavras - 1):
      if(i >= removido):
        vertices[i] = vertices[i+1]
        vertices[i]["indice"] = i

      n_vizinhos = vertices[i]["n_vizinhos"]
      vizinhos = vertices[i]["proximos"]
      posicao = buscaVizinho(n_vizinhos, vizinhos, removido)

      if(posicao != -1):
        aresta = criaAresta(vertices[i]["indice"], removido)
        removeAresta(grafo, n_palavras, vertices, aresta)

    vertices.pop()

### Imprime vértices

In [89]:
def imprimeVertices(network, n_palavras, vertices):
  for i in range(n_palavras):
    network.add_node( # Adiciona vértices
        i, # Índice
        label=vertices[i]["palavra"], # Descrição do vértice
        color="yellow"
    )

### Imprime arestas

In [90]:
def imprimeArestas(network, n_palavras, vertices):
  for i in range(n_palavras):
    vizinhos = vertices[i]["proximos"]
    for j in range(vertices[i]["n_vizinhos"]):
      distancia = float(vizinhos[j]["distancia"])
      if(distancia > 0 and distancia < 1):
        peso = (1-distancia) # Caso seja um valor entre a escala da distância semântica
      else:
        peso = distancia # Caso seja um valor fora do limite
      network.add_edge( # Adiciona as arestas
          vertices[i]["indice"], # Origem
          vizinhos[j]["indice"], # Destino
          value=peso, # Peso
          title=f'''
          {vertices[i]["palavra"]}
          {vizinhos[j]["vizinho"]}
          Peso: {peso:.2f}
                        ''', # Descrição
          color="red"
      )

### Imprime grafo

In [91]:
def imprimeGrafo(n_palavras, vertices):
  net = Network(notebook=True, cdn_resources='remote', directed=False, height="1200px", width="100%", bgcolor="black", font_color="white")
  imprimeVertices(net, n_palavras, vertices)
  imprimeArestas(net, n_palavras, vertices)
  net.barnes_hut() # Dispersa melhor os vértices
  net.show("grafo.html") # Salva o grafo

### Mostra vértice

In [92]:
def mostraVertice(vertices, indice):
  palavra = vertices[indice]["palavra"]
  n_vizinhos = vertices[indice]["n_vizinhos"]
  vizinhos = vertices[indice]["proximos"]
  print(f"\nPalavra: {palavra}\n")
  for i in range(n_vizinhos):
    vizinho =  vizinhos[i]["vizinho"]
    distancia = vizinhos[i]["distancia"]
    print(f'''
    Vizinho: {vizinho}
    Distância: {distancia}''')

# Aplicação

## Código

In [93]:
def menu():
    memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
    fim = False

    while(fim == False):
        print(
    '''
    Menu:

        1) Ler dados do arquivo
        2) Gravar dados no arquivo grafo.txt
        3) Inserir vértice
        4) Inserir aresta
        5) Remover vértice
        6) Remover aresta
        7) Exibir grafo
        8) Exibir matriz
        9) Apresentar a conexidade do grafo
        10) Encerrar a aplicação
        11) Buscar um índice pela palavra
        12) Buscar uma palavra pelo índice
        13) Mostrar informações do vértice
        14) Fazer busca vetorial
    ''')

        choice = input()
        try:
            choice = int(choice)
            if choice == 1: # Cria grafo
                dados = leArquivoHTTP('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/palavras.txt')
                #dados = leArquivo("palavras.txt")
                n_palavras = dados[0]
                vertices = dados[1]
                embedding(memoria, n_palavras, vertices)
                grafo = integraGrafo(memoria, n_palavras, vertices)
                print("Grafo criado com sucesso!")

            elif choice == 2: # Grava dados no arquivo .txt
                try:
                    gravaDados(n_palavras, vertices)
                    print("Os dados foram salvos no arquivo 'grafo.txt'.")
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 3: # Insere vértice
                palavra = input("Palavra a ser inserida: ")
                try:
                    insereVertice(grafo, n_palavras, vertices, palavra)
                    n_palavras += 1
                    print("Vértice inserido com sucesso!")

                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 4: # Insere aresta
                try:
                    origem = int(input("Insira o índice de origem: "))
                    destino = int(input("Insira o índice de destino: "))
                    peso = float(input("Insira o peso: "))
                    aresta = criaAresta(origem, destino)
                    try:
                        insereAresta(grafo, n_palavras, vertices, aresta, peso)
                        print("Aresta inserida com sucesso!")

                    except NameError:
                        print("[Erro: Grafo não criado]")

                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 5: # Remove vértice
                try:
                    indice = int(input("Insira o índice do vértice: "))
                    try:
                        removeVertice(grafo, n_palavras, vertices, indice)
                        n_palavras -= 1
                        print("Vértice removido com sucesso!")

                    except NameError:
                        print("[Erro: Grafo não criado]")

                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 6: # Remove aresta
                try:
                    origem = int(input("Insira o índice de origem: "))
                    destino = int(input("Insira o índice de destino: "))
                    aresta = criaAresta(origem, destino)
                    try:
                        removeAresta(grafo, n_palavras, vertices, aresta)
                        print("Aresta removida com sucesso!")

                    except NameError:
                        print("[Erro: Grafo não criado]")

                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 7: # Exibe grafo
                try:
                    imprimeGrafo(n_palavras, vertices)
                    print(f"\nGrafo não-direcionado rotulado com {grafo.n} vértices e {grafo.m} arestas\n")
                    print("O grafo visual foi criado no arquivo 'grafo.html'.")
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 8: # Exibe matriz
                try:
                    grafo.showMin()
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 9: # Apresenta a conexidade do grafo e grafo reduzido
                try:
                    if(grafo.conexidade()):
                        print("O grafo é conexo.")
                    else:
                        print("O grafo não é conexo.")
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 10: # Encerra
                fim = True
                print("Encerrando programa...")

            elif choice == 11: # Busca um índice pela palavra
                try:
                    palavra = input("Palavra a ser consultada: ")
                    indice = buscaIndice(n_palavras, vertices, palavra)
                    if(indice == -1):
                        print("[Erro: palavra não encontrada]")
                    else:
                        print(f"Índice de {palavra}: {indice}")
                        print(("\nAperte ENTER para continuar "))
                        ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 12: # Busca um índice pela palavra
                try:
                    indice = int(input("Índice a ser consultado: "))
                    try:
                        palavra = buscaPalavra(n_palavras, vertices, indice)
                        if(indice == -1):
                            print("[Erro: palavra não encontrada]")
                        else:
                            print(f"Palavra do índice {indice}: {palavra}")
                            print(("\nAperte ENTER para continuar "))
                            ok = input()
                    except NameError:
                      print("[Erro: Grafo não criado]")
                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 13: # Mostra informações do vértice
                try:
                    indice = int(input("Índice do vértice: "))
                    try:
                        mostraVertice(vertices, indice)
                        print(("\nAperte ENTER para continuar "))
                        ok = input()
                    except NameError:
                      print("[Erro: Grafo não criado]")
                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 14: # Fazer busca vetorial de uma palavra
                busca = input("Insira uma palavra: ")
                try:
                    resultado = memoria.search(busca, top_n=3)
                    print(f"\n\nBusca: {busca}\n")
                    for i in range(3):
                        palavra = resultado[i]['chunk']
                        distancia = resultado[i]['distance']
                        print(f"Palavra: {palavra}\nDistância: {distancia:.2f}\n")
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except IndexError:
                    print("[Erro: Grafo não criado]")

            else:
                print("Opção inválida.")

        except ValueError:
            print("[Erro: a entrada não é do tipo int]")

        #time.sleep(1)

        clear_output(wait=True) # Limpa o terminal caso esteja no Google Colab

## Menu

In [94]:
menu()


    Menu:

        1) Ler dados do arquivo
        2) Gravar dados no arquivo grafo.txt
        3) Inserir vértice
        4) Inserir aresta
        5) Remover vértice
        6) Remover aresta
        7) Exibir grafo
        8) Exibir matriz
        9) Apresentar a conexidade do grafo
        10) Encerrar a aplicação
        11) Buscar um índice pela palavra
        12) Buscar uma palavra pelo índice
        13) Mostrar informações do vértice
        14) Fazer busca vetorial
    


KeyboardInterrupt: Interrupted by user

# Testes

In [113]:
from ipywidgets import widgets
from IPython.display import display, clear_output

# Função para exibir o menu
def exibir_menu():
    menu_text = widgets.HTML(value="""
    <h1>Menu:</h1>
    <ol>
        <li>Ler dados do arquivo</li>
        <li>Gravar dados no arquivo grafo.txt</li>
        <li>Inserir vértice</li>
        <li>Inserir aresta</li>
        <li>Remover vértice</li>
        <li>Remover aresta</li>
        <li>Exibir grafo</li>
        <li>Exibir matriz</li>
        <li>Apresentar a conexidade do grafo</li>
        <li>Encerrar a aplicação</li>
        <li>Buscar um índice pela palavra</li>
        <li>Buscar uma palavra pelo índice</li>
        <li>Mostrar informações do vértice</li>
        <li>Fazer busca vetorial</li>
    </ol>
    """)
    display(menu_text)

# Função para capturar a escolha do usuário
def capturar_escolha(btn):
    clear_output(wait=True)
    escolha = int(entrada.value)
    if escolha == 10:
        print("Encerrando programa...")
        return
    print(f"Você escolheu a opção {escolha}")
    # Adicione aqui a lógica para cada opção do menu
    exibir_menu()  # Exibe o menu novamente após a escolha do usuário
    display(entrada, botao)

# Widgets de entrada e botão
entrada = widgets.Text(placeholder='Digite uma opção...')
botao = widgets.Button(description="Confirmar")
botao.on_click(capturar_escolha)

# Exibir menu e widgets
exibir_menu()
display(entrada, botao)


Text(value='', placeholder='Digite uma opção...')

Button(description='Confirmar', style=ButtonStyle())